# Fake News Classifier using LSTM,Bi-LSTM,Tensorflow 2-GPU

Imports Libraries

In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [46]:
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.text import one_hot

In [3]:
fake_news_data=pd.read_csv(r"C:\Users\SRKT\Downloads\fake-news\train.csv")

Preprocessing Data

In [4]:
fake_news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
#fake_news_data[['title','author']]=fake_news_data[['title','author']].fillna(value="missing")
fake_news_data=fake_news_data.dropna()

In [6]:
fake_news_data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
length=[]
[length.append(len(str(text)))for text in fake_news_data['text']]
fake_news_data['length']=length
fake_news_data.head()

,id,title,author,text,label,length
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,4930
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,4160
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,7692
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,3237
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,938


In [8]:
fake_news_data['length'].min(),fake_news_data['length'].max(),

(1, 142961)

In [9]:
sum(fake_news_data['length'])/len(fake_news_data['length'])

4800.181843040744

In [10]:
max_features=5000

In [11]:
Y=fake_news_data['label']

In [12]:
X=fake_news_data.drop(['id','length','label'],axis=1)

In [13]:
X.head()

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [14]:
X.shape,Y.shape

((18285, 3), (18285,))

In [15]:
msg=X.copy()
msg.reset_index(inplace=True)

In [16]:
msg.head()

,index,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SRKT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Dataset Preprocessing

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(msg)):
    review = re.sub('[^a-zA-Z]', ' ', msg['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [22]:
one_rep=[one_hot(words,max_features)for words in corpus]
one_rep

[[3023, 1846, 1961, 1355, 2680, 1734, 3714, 3023, 1788, 1026],
 [3518, 91, 1076, 442, 798, 1673, 3170],
 [758, 174, 1307, 3757],
 [3998, 3900, 2125, 4839, 135, 3726],
 [2500, 798, 4087, 2766, 255, 1696, 798, 4795, 1042, 2074],
 [4936,
  2718,
  3398,
  1492,
  2985,
  2188,
  54,
  1772,
  2582,
  4686,
  2458,
  632,
  1259,
  2894,
  3170],
 [1830, 3716, 2470, 944, 734, 1192, 4275, 949, 1018, 935, 250],
 [1617, 4019, 259, 4993, 3931, 684, 2188, 2075, 1018, 935, 250],
 [3076, 3427, 1370, 1433, 2450, 1585, 3067, 3793, 2188, 726],
 [2970, 4512, 2529, 1615, 3531, 3251, 2058, 3086],
 [4112, 4322, 4876, 394, 2390, 1016, 3930, 4511, 2568, 2101, 3358],
 [4839, 3010, 2680, 1585, 2188, 3931],
 [3053, 2415, 242, 2742, 2810, 4451, 4301, 4369, 3932],
 [2578, 3664, 1548, 3968, 827, 3072, 4436, 1018, 935, 250],
 [3105, 2886, 4302, 4821, 3905, 1018, 935, 250],
 [2061, 717, 13, 1893, 2850, 1232, 4415, 3497, 680, 2615],
 [3586, 4755, 91],
 [2081, 1167, 3400, 1369, 2188, 2593, 1666, 3170],
 [3345, 564,

In [23]:
sen_len=20
pad_sen=pad_sequences(one_rep,padding='pre',maxlen=sen_len)

In [24]:
pad_sen

array([[   0,    0,    0, ..., 3023, 1788, 1026],
       [   0,    0,    0, ...,  798, 1673, 3170],
       [   0,    0,    0, ...,  174, 1307, 3757],
       ...,
       [   0,    0,    0, ..., 1018,  935,  250],
       [   0,    0,    0, ...,  891, 4234, 3201],
       [   0,    0,    0, ..., 3801,  681, 4352]])

In [25]:
len(pad_sen)

18285

Model Creating

In [38]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(max_features,embedding_vector_features,input_length=sen_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation="relu"))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [39]:
x_final=np.array(pad_sen)
y_final=np.array(Y)

In [40]:
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.30,random_state=42)

Training the Model

In [41]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Train on 12799 samples, validate on 5486 samples
Epoch 1/10
12799/12799 [==============================] - 10s 755us/sample - loss: 0.4750 - accuracy: 0.7662 - val_loss: 0.3700 - val_accuracy: 0.8996
Epoch 2/10
12799/12799 [==============================] - 5s 359us/sample - loss: 0.2106 - accuracy: 0.9303 - val_loss: 0.3143 - val_accuracy: 0.9141
Epoch 3/10
12799/12799 [==============================] - 5s 420us/sample - loss: 0.1596 - accuracy: 0.9482 - val_loss: 0.3170 - val_accuracy: 0.9079
Epoch 4/10
12799/12799 [==============================] - 5s 407us/sample - loss: 0.1359 - accuracy: 0.9618 - val_loss: 0.4402 - val_accuracy: 0.9147
Epoch 5/10
12799/12799 [==============================] - 5s 423us/sample - loss: 0.1369 - accuracy: 0.9643 - val_loss: 0.5493 - val_accuracy: 0.9172
Epoch 6/10
12799/12799 [==============================] - 5s 421us/sample - loss: 0.1151 - accuracy: 0.9734 - val_loss: 0.5467 - val_accuracy: 0.9098
Epoch 7/10
12799/12799 [==========================

Performance Metrics and Accuracy

In [35]:
y_pred=model.predict_classes(x_test)

In [42]:
accuracy_score(y_test,y_pred)

0.9097703244622676

# Using Bidirectional LSTM

In [47]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(max_features,embedding_vector_features,input_length=sen_len))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation="relu"))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Train on 12799 samples, validate on 5486 samples
Epoch 1/10
12799/12799 [==============================] - 6s 477us/sample - loss: 0.4988 - accuracy: 0.7787 - val_loss: 0.2476 - val_accuracy: 0.9074
Epoch 2/10
12799/12799 [==============================] - 2s 180us/sample - loss: 0.2536 - accuracy: 0.8890 - val_loss: 0.2891 - val_accuracy: 0.9085
Epoch 3/10
12799/12799 [==============================] - 2s 186us/sample - loss: 0.1768 - accuracy: 0.9433 - val_loss: 0.3437 - val_accuracy: 0.9172
Epoch 4/10
12799/12799 [==============================] - 2s 181us/sample - loss: 0.1457 - accuracy: 0.9584 - val_loss: 0.4282 - val_accuracy: 0.9191
Epoch 5/10
12799/12799 [==============================] - 2s 186us/sample - loss: 0.1171 - accuracy: 0.9730 - val_loss: 0.5150 - val_accuracy: 0.9174
Epoch 6/10
12799/12799 [==============================] - 2s 186us/sample - loss: 0.1064 - accuracy: 0.9805 - val_loss: 0.6255 - val_accuracy: 0.9136
Epoch 7/10
12799/12799 [===========================